# Data Processing and Analysis
Consists of following major components.
1.   Demand calculation
2.   Identifying the price for each product
3. Delivery cost related calculations




In [1]:
import pandas as pd
import numpy as np
outlet_details = pd.read_excel('Sanitized Test Data - 10_10_2024.xlsx', sheet_name='Outlet Details')
sales_details = pd.read_excel('Sanitized Test Data - 10_10_2024.xlsx', sheet_name='Sales Data')
vehicle_data = pd.read_excel('Sanitized Test Data - 10_10_2024.xlsx', sheet_name='Vehicle Details')
inventory_data = pd.read_excel('Sanitized Test Data - 10_10_2024.xlsx', sheet_name='Inventory Details')

#### Demand


In [2]:
demand_df = sales_details.groupby(['outlet_id', 'product_id'])['qty'].max().reset_index()
demand_df.rename(columns={'qty': 'max_demand'}, inplace=True)
demand_df = demand_df.sort_values(by=['outlet_id'])

In [3]:
demand_2d = demand_df.pivot(index='outlet_id', columns='product_id')
demand_2d = demand_2d.fillna(0)

# Extract the outlet and product mappings
outlet_mapping = {idx: outlet for idx, outlet in enumerate(demand_2d.index)}
product_mapping = {idx: product for idx, product in enumerate(demand_2d.columns)}

# Convert the pivoted DataFrame to a 2D array
demand_array = demand_2d.to_numpy()

# Display the mappings and the array
print("Outlet Mapping:", outlet_mapping)
print("Product Mapping:", product_mapping)
print("Demand Array:\n", demand_array)

Outlet Mapping: {0: 4569, 1: 4604, 2: 5727, 3: 5729, 4: 5801, 5: 5806, 6: 5838, 7: 5903, 8: 5905, 9: 5906, 10: 5907, 11: 5910, 12: 5911, 13: 5912, 14: 5913, 15: 5914, 16: 5917, 17: 5924, 18: 5927, 19: 5930, 20: 5933, 21: 5934, 22: 5940, 23: 5944, 24: 5946, 25: 5947, 26: 5950, 27: 5952, 28: 5954, 29: 5956, 30: 5967, 31: 5969, 32: 5970, 33: 5973, 34: 5976, 35: 5979, 36: 5981, 37: 5991, 38: 5993, 39: 5995, 40: 5996, 41: 5997, 42: 5998, 43: 6002, 44: 6005, 45: 6006, 46: 6007, 47: 6008, 48: 6012, 49: 6014, 50: 6015, 51: 6021, 52: 6022, 53: 6025, 54: 6031, 55: 6033, 56: 6034, 57: 6035, 58: 6036, 59: 6037, 60: 6038, 61: 6039, 62: 6042, 63: 6043, 64: 6044, 65: 6045, 66: 6049, 67: 6055, 68: 6056, 69: 6057, 70: 6063, 71: 6064, 72: 6068, 73: 6073, 74: 6075, 75: 6076, 76: 6077, 77: 6078, 78: 6079, 79: 6083, 80: 6087, 81: 6090, 82: 6092, 83: 6097, 84: 6098, 85: 6099, 86: 6102, 87: 6103, 88: 6104, 89: 6105, 90: 6109, 91: 6115, 92: 6116, 93: 6119, 94: 6120, 95: 6124, 96: 6129, 97: 6135, 98: 6136, 99:

In [4]:
demand_df

outlet_id  product_id  max_demand
0          4569        5064           2
1          4569        5065           2
2          4569        5069           2
3          4569        5070           2
4          4569        5072           2
...         ...         ...         ...
5118    1034910        5078           2
5115    1034910        5070           1
5116    1034910        5076           1
5117    1034910        5077           2
5119    1034910        5079           2

[5120 rows x 3 columns]

#### Price
In here we are considering the latest price

In [5]:
sales_details['invoice_date'] = pd.to_datetime(sales_details['Invoice Date'])

# getting the per item price
sales_details['price'] = sales_details['value'] / sales_details['qty']

sales_details = sales_details.sort_values(by=['product_id', 'Invoice Date'], ascending=[True, False])
latest_price_df = sales_details.drop_duplicates(subset=['product_id'], keep='first')


latest_price_df = latest_price_df[['product_id', 'price']].set_index('product_id')

# order by product_id
latest_price_df = latest_price_df.sort_values(by=['product_id'])

# create a list of price
price_list = latest_price_df['price'].tolist()

In [6]:
price_list

[504.9,
 471.24,
 1009.8,
 826.2,
 866.592,
 454.41,
 1211.76,
 1161.27,
 1161.27,
 888.624,
 218.79,
 1683.0,
 328.185,
 1262.25,
 403.92,
 403.92]

#### Distance


In [7]:
from math import radians, sin, cos, sqrt, atan2

def haversine(lat1, lon1, lat2, lon2):
    # Radius of Earth in kilometers
    R = 6371.0

    # Convert latitude and longitude from degrees to radians
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    # Compute differences
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    # Apply the Haversine formula
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    # Distance in kilometers
    distance = R * c

    return distance

In [8]:
# Pivot point coordinates
pivot_lat, pivot_long = 6.796856130578927, 79.89603028506602

# convert gps_latitude, gps_longitude to float values
outlet_details['gps_latitude'] = outlet_details['gps_latitude'].astype(float)
outlet_details['gps_longitude'] = outlet_details['gps_longitude'].astype(float)

# calculate the distance using haversine function from pivot point to every outlet
outlet_details['distance_from_pivot'] = outlet_details.apply(lambda row: haversine(pivot_lat, pivot_long, row['gps_latitude'], row['gps_longitude']), axis=1)

# keep outlet_id, distance_from_pivot columns
distance = outlet_details[['outlet_id', 'distance_from_pivot']]

# remove duplicate outlet_id
distance = distance.drop_duplicates(subset=['outlet_id'])

# set outlet_id as index
distance = distance.set_index('outlet_id')


# convert the distance_from_pivot column in to a list
distance_array = distance['distance_from_pivot'].tolist()


#### Inventoy Cost

In [9]:
# convert the df to a list
inventory_cost = inventory_data['unit_cost'].tolist()

# Optimization Problem

In [10]:
# !python /Applications/CPLEX_Studio2211/python/setup.py install

In [11]:
import docplex.mp
# first import the Model class from docplex.mp
from docplex.mp.model import Model

# create one model instance, with a name
model = Model(name='Revenue_Maximization')

In [12]:
# by default, all variables in Docplex have a lower bound of 0 and infinite upper bound
# y[j]: Binary variable indicating if shop j is reached
y = model.binary_var_list(keys=675, name="y")
#

# q[i, j]: Quantity of product i delivered to shop j as an integer variable
q = model.integer_var_matrix(keys1=16, keys2=675, name="q",lb=0)

In [13]:
# random data generation
Q = 100000
n = 675 # number of stores
m = 16 # number of products

D = demand_array
# D[0] = [0]*16

Tmax = 5

IC = inventory_cost 
# C = np.random.randint(10, 51, size=n)
C = distance_array

# A = np.random.randint(20, 151, size=(m, n))
A = price_list
Pr = np.random.randint(1, 6, size=n)

In [14]:
D[0][0]

np.float64(0.0)

In [15]:
# Vehicle Capacity Constraint
## for each outlet the sum of q has to be less than the total capacity of vehicles served for that outlet // change this part
model.add_constraint(model.sum(q)<= Q, "total_qty_constraint")

# # Qty Satisfaction Constraint
for i in range(m): # i: product
    for j in range(n): # j: store
        model.add_constraint(q[i, j] <= Q*y[j], f"demand_constraint_{i}_{j}") # store -> product

# # Demand Satisfaction Constraint
for i in range(m): # i: product
    for j in range(n): # j: store
        model.add_constraint(q[i, j] <= D[j][i], f"demand_constraint_{i}_{j}") # store -> product


In [16]:
model

docplex.mp.Model['Revenue_Maximization']

In [17]:
# Objective function components
revenue = model.sum(A[i] * Pr[j] * q[i, j] for i in range(m) for j in range(n))
delivery_costs = model.sum(C[j] * y[j] for j in range(n))
inventory_cost = model.sum(IC[i]*q[i, j] for i in range(m) for j in range(n))

Z = revenue - delivery_costs-inventory_cost
# Define objective function
model.maximize(Z)


In [18]:
model.print_information()

Model: Revenue_Maximization
 - number of variables: 11475
   - binary=675, integer=10800, continuous=0
 - number of constraints: 21601
   - linear=21601
 - parameters: defaults
 - objective: maximize
 - problem type is: MILP


In [19]:
s = model.solve()
model.print_solution()

objective: 49651637.225
status: OPTIMAL_SOLUTION(2)
  y_0=1
  y_1=1
  y_2=1
  y_3=1
  y_4=1
  y_5=1
  y_6=1
  y_7=1
  y_8=1
  y_9=1
  y_10=1
  y_11=1
  y_12=1
  y_13=1
  y_14=1
  y_15=1
  y_16=1
  y_17=1
  y_18=1
  y_19=1
  y_20=1
  y_21=1
  y_22=1
  y_23=1
  y_24=1
  y_25=1
  y_26=1
  y_27=1
  y_28=1
  y_29=1
  y_30=1
  y_31=1
  y_32=1
  y_33=1
  y_34=1
  y_35=1
  y_36=1
  y_37=1
  y_38=1
  y_39=1
  y_40=1
  y_41=1
  y_42=1
  y_43=1
  y_44=1
  y_45=1
  y_46=1
  y_47=1
  y_48=1
  y_49=1
  y_50=1
  y_51=1
  y_52=1
  y_53=1
  y_54=1
  y_55=1
  y_56=1
  y_57=1
  y_58=1
  y_59=1
  y_60=1
  y_61=1
  y_62=1
  y_63=1
  y_64=1
  y_65=1
  y_66=1
  y_67=1
  y_68=1
  y_69=1
  y_70=1
  y_71=1
  y_72=1
  y_73=1
  y_74=1
  y_75=1
  y_76=1
  y_77=1
  y_78=1
  y_79=1
  y_80=1
  y_81=1
  y_82=1
  y_83=1
  y_84=1
  y_85=1
  y_86=1
  y_87=1
  y_88=1
  y_89=1
  y_90=1
  y_91=1
  y_92=1
  y_93=1
  y_94=1
  y_95=1
  y_96=1
  y_97=1
  y_98=1
  y_99=1
  y_100=1
  y_101=1
  y_102=1
  y_103=1
  y_104=1
  y_105=

In [20]:
import numpy as np

# Define the dimensions
n = 675  # Number of outlets
m = 16   # Number of products

# Initialize a NumPy array filled with zeros
outlets_products = np.zeros((n, m), dtype=int)

# Assuming s.iter_variables() returns an iterable of variable objects
for var in s.iter_variables():
    if var.name.startswith('q_'):
        outlet_number = int(var.name.split('_')[2])  # Extract outlet number
        product_id = int(var.name.split('_')[1])      # Extract product ID
        product_qty = int(var.solution_value)          # Get product quantity
        
        # Add the quantity to the corresponding position in the NumPy array
        outlets_products[outlet_number][product_id] += product_qty

print(outlets_products)

[[0 2 2 ... 2 0 0]
 [0 2 1 ... 2 0 0]
 [0 2 2 ... 0 0 0]
 ...
 [0 1 0 ... 1 0 0]
 [0 0 1 ... 0 0 0]
 [0 0 0 ... 2 0 0]]


In [21]:
outlet_list = sorted(int(i) for i in sales_details['outlet_id'].unique())
product_list = sorted(int(i) for i in sales_details['product_id'].unique())
df = pd.DataFrame(outlets_products)
df.columns = product_list
df.index = outlet_list


In [22]:
df

4592  5064  5065  5066  5068  5069  5070  5071  5072  5073  5076  \
4569        0     2     2     0     0     2     2     0     2     0     0   
4604        0     2     1     0     0     2     0     0     1     0     2   
5727        0     2     2     0     0     0     0     0     0     0     0   
5729        0     1     1     0     0     2     1     0     0     0     1   
5801        0     2     0     0     2     3     3     0     2     0     2   
...       ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
1033681     0     0     0     0     0     2     1     0     0     0     0   
1033682     0     4     5     0    99     2     3     0     7     0     1   
1033683     0     1     0     0     0     2     0     0     1     0     2   
1033684     0     0     1     0     0     2     0     0     1     0     2   
1034910     0     0     0     0     0     0     1     0     0     0     1   

         5077  5078  5079  5470  5471  
4569        2     2     2     0     0  
4604        1     2     2     0     0  
5727        1     0     0     0     0  
5729        2     2     2     0     0  
5801        0     2     0     0     0  
...       ...   ...   ...   ...   ...  
1033681     0     0     0     0     0  
1033682     1     1     2     2     0  
1033683     1     2     1     0     0  
1033684     1     1     0     0     0  
1034910     2     2     2     0     0  

[675 rows x 16 columns]

In [23]:
import pandas as pd
import plotly.express as px



# Create the map
fig = px.scatter_mapbox(
    outlet_details,
    lat="gps_latitude",
    lon="gps_longitude",
    hover_name="outlet_id",  # Show outlet ID when hovering over points
    zoom=10,  # Initial map zoom level
    mapbox_style="carto-positron"  # Map style for a clean background
)

# Center the map around the average latitude and longitude values for better visualization
fig.update_layout(
    title="Outlet Locations Map",
    mapbox=dict(center=dict(lat=6.80423464, lon=79.95780262), zoom=12)
)

# Show the plot
fig.show()

In [24]:
# Create a click event for the outlets
def get_product_quantities(outlet_id):
    return df.loc[outlet_id].tolist()

In [35]:
import folium

# Function to adjust costs based on environmental factors
def adjust_costs(cost_matrix, environmental_data):
    adjusted_matrix = cost_matrix.copy()
    for (i, j), factor in environmental_data.items():
        adjusted_matrix[i][j] *= factor
        adjusted_matrix[j][i] *= factor
    return adjusted_matrix

# Function to create a distance matrix for TSP using Haversine distance
def haversine_distance(coord1, coord2):
    # Earth radius in kilometers
    R = 6371.0
    lat1, lon1 = np.radians(coord1)
    lat2, lon2 = np.radians(coord2)
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c

def calculate_distance_matrix(locations):
    num_locations = len(locations)
    distance_matrix = np.zeros((num_locations, num_locations))
    for i in range(num_locations):
        for j in range(num_locations):
            if i != j:
                distance_matrix[i][j] = haversine_distance(locations[i], locations[j])
    return distance_matrix

# Function to solve TSP
def solve_tsp(locations, environmental_data, start_index=0):
    mdl = Model(name="TSP_with_environmental_factors")
    # Calculate the initial distance matrix
    distance_matrix = calculate_distance_matrix(locations)
    # Adjust the cost matrix based on environmental factors
    adjusted_matrix = adjust_costs(distance_matrix, environmental_data)

    print(len(adjusted_matrix))
    
    num_locations = len(locations)
    # Decision variables: x[i, j] is 1 if the route goes from i to j
    x = mdl.binary_var_matrix(num_locations, num_locations, name="x")
    # Objective: Minimize the total distance
    mdl.minimize(mdl.sum(adjusted_matrix[i][j] * x[i, j] for i in range(num_locations) for j in range(num_locations) if i != j))
    
    # Constraints: Each location must be visited exactly once
    for i in range(num_locations):
        mdl.add_constraint(mdl.sum(x[i, j] for j in range(num_locations) if j != i) == 1, f"depart_{i}")
        mdl.add_constraint(mdl.sum(x[j, i] for j in range(num_locations) if j != i) == 1, f"arrive_{i}")
    
    mdl.add_constraint(mdl.sum(x[start_index, j] for j in range(num_locations) if j != start_index) == 1, f"start_from_{start_index}")
    mdl.add_constraint(mdl.sum(x[i, start_index] for i in range(num_locations) if i != start_index) == 1, f"end_at_{start_index}")
    
    # Subtour elimination constraints using MTZ formulation
    u = mdl.integer_var_list(num_locations, lb=0, ub=num_locations - 1, name="u")
    for i in range(1, num_locations):
        for j in range(1, num_locations):
            if i != j:
                mdl.add_constraint(u[i] - u[j] + num_locations * x[i, j] <= num_locations - 1)
    
    # Set CPLEX parameters to improve performance
    mdl.context.cplex_parameters.timelimit = 300  # Time limit in seconds (5 minutes)
    mdl.context.cplex_parameters.threads = 12
    # Solve the model
    solution = mdl.solve(log_output=True)
    print("finishhhhhhh")
    # get the route
    route = [(i, j) for i in range(num_locations) for j in range(num_locations) if i != j and x[i, j].solution_value > 0.5]
    
    return route, locations

# Plotting function for the solution on a map
def plot_route_on_map(route, locations, start_location):
    # Initialize a folium map centered around the start location
    map_ = folium.Map(location=start_location, zoom_start=12)
    print(len(locations))
    
    # Add markers for each location
    for idx, (lat, lon) in enumerate(locations):
        folium.Marker(location=(lat, lon), popup=f"Location {idx}").add_to(map_)
        print(idx)
    
    # Draw the route
    for (i, j) in route:
        folium.PolyLine(
            locations=[locations[i], locations[j]],
            color="blue",
            weight=2.5,
            opacity=1
        ).add_to(map_)
    
    return map_

# Main execution
if __name__ == "__main__":
    # get location per beat ID
    locations_in = outlet_details[outlet_details['beat'] == 8][['gps_latitude', 'gps_longitude']].values.tolist()
    # print(len(locations))
    # environmental_data = {(35, 12): 0}
    environmental_data = {}
    route, locations = solve_tsp(locations_in, environmental_data)
    # print(len(locations))
    start_location = locations[0]  # Set the starting location for the map
    map_ = plot_route_on_map(route, locations, start_location)
    
    # Display the map
    map_.save("route_beat_8.html")  # Save the map as an HTML file


45
Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 12
CPXPARAM_TimeLimit                               300
Tried aggregator 1 time.
MIP Presolve eliminated 2 rows and 46 columns.
Reduced MIP has 1982 rows, 2024 columns, and 9636 nonzeros.
Reduced MIP has 1980 binaries, 44 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (5.34 ticks)
Probing time = 0.01 sec. (3.46 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 1982 rows, 2024 columns, and 9636 nonzeros.
Reduced MIP has 1980 binaries, 44 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (6.98 ticks)
Probing time = 0.00 sec. (3.45 ticks)
Clique table members: 1030.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 12 threads.
Root relaxation solution time = 0.00 sec. (3.23 ticks)

        Nodes                

In [26]:
outlet_details

vehicle_no  beat  outlet_id  gps_latitude  gps_longitude  \
0             1     1       5967      6.815389      79.971873   
1             1     1       5969      6.814624      79.972596   
2             1     1       5970      6.814357      79.972578   
3             1     1       5973      6.811957      79.971245   
4             1     1       5976      6.814357      79.972578   
..          ...   ...        ...           ...            ...   
697           3    18     705014      6.772537      79.993922   
698           3    18     712608      6.771243      79.979241   
699           3    18     720304      6.775457      79.984774   
700           3    18    1024941      6.811867      79.956049   
701           3    18    1030696      6.773957      79.982737   

     distance_from_pivot  
0               8.623791  
1               8.681529  
2               8.672937  
3               8.472683  
4               8.672937  
..                   ...  
697            11.141945  
698             9.619165  
699            10.083511  
700             6.833751  
701             9.906627  

[702 rows x 6 columns]

In [27]:
print(start_location)

[6.8153894, 79.9718734]
